In [127]:
import os
from rdkit import Chem
import openbabel as ob
import glob
import json
import numpy as np
import pandas as pd
from sklearn.preprocessing import StandardScaler
from tqdm import tqdm
obConversion = ob.OBConversion()
_ = obConversion.SetInAndOutFormats("xyz", "mol2")

In [20]:
train = pd.read_csv('../input/train.csv')
ss = StandardScaler()
train['scc_scaled'] = ss.fit_transform(train[['scalar_coupling_constant']])
mean_scc = train.groupby('molecule_name')['scc_scaled'].mean()

In [158]:
def read_xyz(mol, index_of_mu = 4):
    file_path = f'/home/robmulla/Repos/gated-graph-neural-network-samples/data/qm9_raw/{mol}.xyz'
    with open(file_path, 'r') as f:
        lines = f.readlines()
        smiles = lines[-2].split('\t')[0]
        properties = lines[1].split('\t')
        mu = float(properties[index_of_mu])
        #average_scc = mean_scc[mol]
        ob_mol = ob.OBMol()
        obConversion.ReadFile(ob_mol, file_path)
        rdkit_mol = Chem.MolFromSmiles(smiles)
        rdkit_mol = Chem.AddHs(rdkit_mol)
    # return {'smiles': smiles, 'mu': mu, 'average_scc': average_scc, 'ob_mol': ob_mol, 'rdkit_mol': rdkit_mol}
    return {'smiles': smiles, 'mu': mu, 'ob_mol': ob_mol, 'rdkit_mol': rdkit_mol}

In [160]:
train.loc[train['molecule_name'] == 'dsgdb9nsd_133884'].head()

,id,molecule_name,atom_index_0,atom_index_1,type,scalar_coupling_constant,scc_scaled
4658069,4658069,dsgdb9nsd_133884,9,0,1JHC,103.632000,2.510172
4658070,4658070,dsgdb9nsd_133884,9,1,2JHN,2.096360,-0.395664
4658071,4658071,dsgdb9nsd_133884,9,2,3JHC,5.046370,-0.311238
4658072,4658072,dsgdb9nsd_133884,9,5,3JHC,-0.459722,-0.468816
4658073,4658073,dsgdb9nsd_133884,9,6,2JHC,-0.035525,-0.456676


In [161]:
test = pd.read_csv('../input/test.csv')

In [166]:
!rm angles_train.csv
count = 0
with open('angles_train.csv', mode='a') as file_:
    # Write the CSV header
    file_.write('molecule_name,atom_index_0,atom_index_1,ang0,ang1,ang2,ang3,ang4,ang5,ang6,ang7,ang8,ang9,ang10,ang11,ang12,ang13,ang14,ang15,ang16,ang17,ang18,ang19,ang20,ang21,ang22,ang23,ang24,ang25,ang26,ang27,ang28')
    file_.write("\n")

for mol_name in tqdm(train['molecule_name'].unique()):
    dat = read_xyz(mol_name)
    for atom1 in ob.OBMolAtomIter(dat['ob_mol']):
        for atom0 in ob.OBMolAtomIter(dat['ob_mol']):
            anglist = []
            for atom1 in ob.OBMolAtomIter(dat['ob_mol']):
                ang = atom0.GetAngle(atom1, atom2)
                ang /= 180
                anglist.append(ang)
            atom_0 = atom0.GetIdx() - 1
            atom_1 = atom1.GetIdx() - 1
            anglist.extend([0] * (max_size - len(anglist))) # Extend ang list
            anglist_commas = str(anglist)[1:-1]
            csv_line = f'{mol_name},{atom_0},{atom_1},{anglist_commas}'
            with open('angles_train.csv', mode='a') as file_:
                file_.write(csv_line)
                file_.write("\n")
#     if count == 100:
#         break
#     count += 1

!rm angles_test.csv
with open('angles_test.csv', mode='a') as file_:
    # Write the CSV header
    file_.write('molecule_name,atom_index_0,atom_index_1,ang0,ang1,ang2,ang3,ang4,ang5,ang6,ang7,ang8,ang9,ang10,ang11,ang12,ang13,ang14,ang15,ang16,ang17,ang18,ang19,ang20,ang21,ang22,ang23,ang24,ang25,ang26,ang27,ang28')
    file_.write("\n")

count = 0
for mol_name in tqdm(test['molecule_name'].unique()):
    dat = read_xyz(mol_name)
    for atom1 in ob.OBMolAtomIter(dat['ob_mol']):
        for atom0 in ob.OBMolAtomIter(dat['ob_mol']):
            anglist = []
            for atom1 in ob.OBMolAtomIter(dat['ob_mol']):
                ang = atom0.GetAngle(atom1, atom2)
                ang /= 180
                anglist.append(ang)
            atom_0 = atom0.GetIdx() - 1
            atom_1 = atom1.GetIdx() - 1
            anglist.extend([0] * (max_size - len(anglist))) # Extend ang list
            anglist_commas = str(anglist)[1:-1]
            csv_line = f'{mol_name},{atom_0},{atom_1},{anglist_commas}'
            with open('angles_test.csv', mode='a') as file_:
                file_.write(csv_line)
                file_.write("\n")
#     if count == 100:
#         break
#     count += 1

  0%|          | 93/45772 [00:00<10:07, 75.20it/s] 


In [167]:
pd.read_csv('angles_test.csv')

,molecule_name,atom_index_0,atom_index_1,ang0,ang1,ang2,ang3,ang4,ang5,ang6,...,ang19,ang20,ang21,ang22,ang23,ang24,ang25,ang26,ang27,ang28
0,dsgdb9nsd_000004,0,3,0.000000,0.343739,0.273278,0.447727,0.000000,0.000000,0.000000,...,0.0,0,0,0,0,0,0,0,0,0
1,dsgdb9nsd_000004,1,3,0.551498,0.000000,0.273278,0.447727,0.000000,0.000000,0.000000,...,0.0,0,0,0,0,0,0,0,0,0
2,dsgdb9nsd_000004,2,3,0.551498,0.656261,0.000000,0.447727,0.000000,0.000000,0.000000,...,0.0,0,0,0,0,0,0,0,0,0
3,dsgdb9nsd_000004,3,3,0.448502,0.343739,0.273278,0.000000,0.000000,0.000000,0.000000,...,0.0,0,0,0,0,0,0,0,0,0
4,dsgdb9nsd_000004,0,3,0.000000,0.343739,0.273278,0.447727,0.000000,0.000000,0.000000,...,0.0,0,0,0,0,0,0,0,0,0
5,dsgdb9nsd_000004,1,3,0.551498,0.000000,0.273278,0.447727,0.000000,0.000000,0.000000,...,0.0,0,0,0,0,0,0,0,0,0
6,dsgdb9nsd_000004,2,3,0.551498,0.656261,0.000000,0.447727,0.000000,0.000000,0.000000,...,0.0,0,0,0,0,0,0,0,0,0
7,dsgdb9nsd_000004,3,3,0.448502,0.343739,0.273278,0.000000,0.000000,0.000000,0.000000,...,0.0,0,0,0,0,0,0,0,0,0
8,dsgdb9nsd_000004,0,3,0.000000,0.343739,0.273278,0.447727,0.000000,0.000000,0.000000,...,0.0,0,0,0,0,0,0,0,0,0
9,dsgdb9nsd_000004,1,3,0.551498,0.000000,0.273278,0.447727,0.000000,0.000000,0.000000,...,0.0,0,0,0,0,0,0,0,0,0


In [78]:
sublist + np.zeros(28)

ValueError: operands could not be broadcast together with shapes (18,) (28,) 

In [104]:
max_size = 29
ang_dim = 29
good_ang_mat  = np.zeros((max_size, max_size, ang_dim))

In [105]:
pd.DataFrame(anglist)

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17
0,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[0.02562345176980362, 0.0, 85.59853301467585, ...","[0.02562345176980362, 46.926065155241936, 0.0,...","[0.02562345176980362, 25.41878019118615, 23.39...","[0.02562345176980362, 24.552916858989537, 32.1...","[0.02562345176980362, 27.39523200909474, 43.80...","[0.02562345176980362, 50.85833368165414, 86.23...","[0.02562345176980362, 46.92634747115909, 61.29...","[0.02562345176980362, 25.418827653666295, 37.4...","[0.02562345176980362, 39.7525570263097, 58.313...","[0.02562345176980362, 39.752523578507066, 18.0...","[0.02562345176980362, 37.36758890463138, 56.07...","[0.02562345176980362, 17.850416711935562, 9.70...","[0.02562345176980362, 18.747168773691634, 22.7...","[0.02562345176980362, 22.348847082090227, 35.0...","[0.02562345176980362, 22.34883158853952, 36.69...","[0.02562345176980362, 37.36784624751772, 42.96...","[0.02562345176980362, 17.850449987866646, 27.8..."
1,"[0.0, 0.02562345176980362, 47.475401830082234,...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[46.926065155241936, 0.02562345176980362, 0.0,...","[25.41878019118615, 0.02562345176980362, 33.45...","[24.552916858989537, 0.02562345176980362, 24.9...","[27.39523200909474, 0.02562345176980362, 25.97...","[50.85833368165414, 0.02562345176980362, 51.41...","[46.92634747115909, 0.02562345176980362, 49.48...","[25.418827653666295, 0.02562345176980362, 36.6...","[39.7525570263097, 0.02562345176980362, 38.410...","[39.752523578507066, 0.02562345176980362, 27.9...","[37.36758890463138, 0.02562345176980362, 39.18...","[17.850416711935562, 0.02562345176980362, 23.8...","[18.747168773691634, 0.02562345176980362, 17.6...","[22.348847082090227, 0.02562345176980362, 22.4...","[22.34883158853952, 0.02562345176980362, 18.07...","[37.36784624751772, 0.02562345176980362, 29.17...","[17.850449987866646, 0.02562345176980362, 27.0..."
2,"[0.0, 47.475401830082234, 0.02562345176980362,...","[85.59853301467585, 0.0, 0.02562345176980362, ...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[23.39835133236421, 33.454991758679796, 0.0256...","[32.17340205116068, 24.93485628705758, 0.02562...","[43.807376506361166, 25.970338266971694, 0.025...","[86.23464232398204, 51.41262360988413, 0.02562...","[61.299002039179115, 49.48055735890015, 0.0256...","[37.45602088942241, 36.69655386456817, 0.02562...","[58.313016563472, 38.41049431502801, 0.0256234...","[18.012619912234072, 27.961574694771627, 0.025...","[56.07226066478424, 39.18399941200052, 0.02562...","[9.704874895554644, 23.888128587159468, 0.0256...","[22.747251079614305, 17.636770116749656, 0.025...","[35.08696616590271, 22.42905280163868, 0.02562...","[36.69517948709027, 18.07686900122139, 0.02562...","[42.96408947602385, 29.17669421341217, 0.02562...","[27.89158164229956, 27.097921147945165, 0.0256..."
3,"[0.0, 45.71578717963194, 16.499044099226396, 0...","[108.86543262918192, 0.0, 32.23313043004983, 0...","[140.10260456840945, 114.3118778112704, 0.0, 0...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[66.35305109358637, 46.702258353825364, 36.872...","[75.23197510434483, 49.47945279085225, 34.8302...","[121.07010032695126, 73.15306888124837, 39.319...","[100.80830819130246, 81.0583453585899, 40.0629...","[77.211663042242, 72.98696691283126, 40.062975...","[53.4755838255377, 37.81053356267945, 5.833920...","[32.823378119428675, 29.276302345286812, 15.32...","[90.80940968797803, 69.74844048743616, 35.0654...","[25.47983648645328, 36.81282449467177, 32.0953...","[40.63144735123838, 26.003695761910137, 20.405...","[58.47752139925416, 39.366575938542674, 25.206...","[62.048891758326164, 39.74025833169798, 29.367...","[69.26866796702654, 50.14011219519801, 26.6822...","[50.923668372395326, 50.01398916174614, 23.772..."
4,"[0.0, 65.13212596240152, 37.68022724204026, 24...","[90.31495717860894, 0.0, 40.89900411745019, 25...","[110.14637070679908, 114.16613959549221, 0.0, ...","[89.33250714166225, 107.67660974637379, 105.